In [1]:
import numpy as np

import os

import torch
import torch.nn as nn
import torch.optim as optim

device = 'cuda'

TRAIN_COUNT = 60
VALIDATION_COUNT = 20
TEST_COUNT = 20

In [ ]:
# Step 1: Data Preparation
dataPath = "../sensorium_data/"
mousePath = "dynamic29515-10-12-Video-9b4f6a1a067fe51e15306b9628efea20"

# [index][XYZ]
coordinates = np.load(dataPath + mousePath + "/meta/neurons/cell_motor_coordinates.npy")
print(coordinates.shape)

# responses = []
# responsesPath = dataPath + mousePath + "/data/responses/"
# for fileName in os.listdir(responsesPath):
#     responses.append(np.load(responsesPath + fileName))

[index][time]
responses = np.load(dataPath + mouseKey + "/data/responses/0.npy")
print(responses.shape)

# [height][width][time]
videos = np.load(dataPath + mouseKey + "/data/videos/0.npy")
print(videos.shape)

neuronCount = responses.shape[0]
frameCount = videos.shape[2]

(7863, 3)


In [ ]:
# Step 2: Model Definition
import Conv4d from Conv4d

class UNet(nn.Module):
    def __init__(self, inputChannelCount: int, outputChannelCount: int):
        super(UNet, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            # nn.Conv2d(inputChannelCount, 64, kernel_size=3, padding=1),
            Conv4d(inputChannelCount, 64, kernel_size=3, padding=1)
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
        )

        # Final output layer
        self.out = nn.Conv2d(64, outputChannelCount, kernel_size=1)

    def forward(self, x):
        # Encoder
        x1 = self.encoder(x)
        # Decoder
        x = self.decoder(x1)
        # Concatenate skip connection from encoder
        x = torch.cat([x, x1], dim=1)
        # Output layer
        x = self.out(x)
        return x

model = UNet()

In [11]:
# Step 3: Loss Function
criterion = nn.MSELoss()

7863

In [ ]:
# Step 4: Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Step 5: Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:  # Assuming train_loader is DataLoader
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print average loss per epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

In [ ]:
# Step 6: Validation/Test
model.eval()
with torch.no_grad():
    for inputs, labels in val_loader:  # Assuming val_loader is DataLoader
        outputs = model(inputs)
        # Evaluate model performance (e.g., compute accuracy, loss, etc.)